# Bringing your own GPU
This predictor demonstrates how to bring your own GPU for making predictions on Function. Function integrates with [Runhouse](https://www.run.house/) to send your predictor code to your own custom GPU instances.

> For startups, you'll find this especially useful if you have credits with the major cloud providers (e.g. AWS, Azure, GCP).

> For enteprirse workloads, we instead recommend bringing your own cluster or running Function on-prem. [Reach out](mailto:sales@fxn.ai) to learn more.

## Installing Runhouse
First, we'll need to install `rsync`:

In [ ]:
!apt-get install -y rsync

Now let's install Runhouse:

In [ ]:
# Install Runhouse
%pip install runhouse

To login to RH, we'll need a Runhouse token. Head over to [your Runhouse account](https://www.run.house/account) to generate your RH token, then create a Function environment variable with your Runhouse token:
```bash
# Open a terminal and run the following command:
fxn env create RH_TOKEN <Your Runhouse token>
```

## Logging into Runhouse

In [ ]:
from os import environ
import runhouse as rh

is_in_fxn = "RH_TOKEN" in environ

if is_in_fxn:
    rh.login(
        token=environ["RH_TOKEN"],
        download_secrets=True,
        download_config=True,
        upload_config=True,
        upload_secrets=True,
        interactive=False
    )

> Note: We've added a check to ensure that this code isn't executed on our Runhouse worker. It should only be executed in our Function predictor which creates the worker on Runhouse.

## Creating an On-Demand Cluster
Now, we can create a cluster on our AWS account:

In [ ]:
# Create an instance on AWS
gpu = rh.cluster(
    name="rh-a10x",
    instance_type="g5.2xlarge",
    provider="aws",
    autostop_mins=5,
    region="us-east-1"
).save() if is_in_fxn else None

## Implementing the Predictor
Let's create a predictor that uses StableDiffusion to generate an image from a text `prompt`. Unlike the standard `stable-diffusion` sample, here will be running the inference on a GPU in our own AWS account:

In [ ]:
def stable_diffusion_generate (prompt: str):
    from diffusers import StableDiffusionPipeline
    from torch import float16
    # Create the SD pipeline on the GPU in our AWS account!!
    pipeline_name = "stabilityai/stable-diffusion-2-base"
    pipeline = StableDiffusionPipeline.from_pretrained(pipeline_name)
    pipeline.to("cuda")
    # Generate an image
    result = pipeline(prompt).images[0]
    # Return
    return result

if is_in_fxn:
    # Send the `stable_diffusion_generate` function to our GPU cluster using RH
    stable_diffusion_generate = rh.function(fn=stable_diffusion_generate).to(
        gpu,
        env=["./", "torch", "diffusers", "transformers"]
    )

The `stable_diffusion_generate` function will be sent to our GPU instance for inference. With this, we can now write our Function predictor:

In [ ]:
# Define the Function predictor
def predict (prompt: str):
    # Generate an image by sending to our custom cluster
    result = stable_diffusion_generate(prompt)
    # Return the result
    return result

## Creating the Predictor on Function
Because we're bringing our own GPU, we can create the predictor on Function and use CPU acceleration:
```bash
fxn create @username/byo-gpu bring-your-gpu.ipynb --overwrite
```

> Replace `username` with your Function username.